# Análise Univariada e Outliers

Abordaremos alguns dos fundamentos da descrição de dados, uma variável de cada vez, chamada **análise univariada**, e esse conhecimento será aplicado à análise de outliers.

Lembre-se de que a análise exploratória de dados pode ser demorada, especialmente à medida que você analisa cada variável. Buracos nos dados podem frequentemente levar você a caminhos que consomem horas, até mesmo dias ou semanas, mas, por uma questão de conveniência, mostraremos esse processo com algumas variáveis ​​que hipotetizamos serem relevantes para colisões com pássaros.

Vamos começar trazendo os dados da última seção.

In [ ]:
import pandas as pd 

url = r"https://github.com/thomasnield/anaconda_python_eda/raw/public/birdstrike_section2.csv"
df = pd.read_csv(url, index_col='INDEX_NR', parse_dates=["INCIDENT_DATE"])
with pd.option_context('display.max_columns', None):
  display(df)

Vamos também cuidar de algumas conversões de tipos de dados que não são salvas no CSV.

In [ ]:
# Transforme PHASE_OF_FLIGHT em uma categoria
phase_of_flt = pd.CategoricalDtype(categories=['Parked', 'Taxi','Take-off Run', 'Approach', 'Departure', 'Climb', 'En Route',
                                               'Descent', 'Landing Roll', 'Arrival', 'Local'])

df["PHASE_OF_FLIGHT"] = df["PHASE_OF_FLIGHT"].astype(phase_of_flt)

# Transformar TIME em tipo timedelta
df["TIME"] = pd.to_timedelta(df["TIME"])

## Variável de Altura

Vamos começar com algumas teorias sobre algumas das variáveis. Talvez a variável `HEIGHT` (a altitude) possa ser relevante para a ocorrência ou não de colisões com pássaros. Afinal, os pássaros precisam pousar para comer e cuidar de seus ninhos. Podemos chamar a função `hist()` nesta coluna para criar um histograma.

In [ ]:
df["HEIGHT"].hist(bins=10)

Ok, isso é interessante. Parece que as colisões com pássaros se distorcem bastante em altitudes mais baixas. Vamos aumentar o número de caixas para obter mais resolução. Não queremos ter muitas caixas porque não temos uma quantidade infinita de dados e, portanto, encontraremos um retorno decrescente e, em seguida, uma perda de informações.

In [ ]:
df["HEIGHT"].hist(bins=30)

A maioria das colisões com pássaros ocorre predominantemente abaixo de 300 metros. Isso faz sentido porque os pássaros, embora frequentemente estejam no ar, voam em grande parte perto do solo. Observe que você também pode construir um histograma diretamente com `matplotlib`. Isso nos permite trazer mais detalhes para o gráfico, como rotular as contagens para cada barra.

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

values, bins, bars = plt.hist(df['HEIGHT'], bins=30, edgecolor='white')
plt.xlabel("HEIGHT (Feet)")
plt.ylabel("# BIRD STRIKES")
plt.title('Height vs Bird Strike Incidents')
plt.bar_label(bars, fontsize=10, color='navy')
plt.margins(x=0.01, y=0.1)
plt.show()

Você pode detectar uma assimetria comparando a **média** (a média da amostra) e a **mediana** (valor mais central da amostra) de uma determinada variável. Se as duas forem muito diferentes, temos uma variável altamente assimétrica, como pode ser visualmente observado acima.

In [ ]:
height_mean = df["HEIGHT"].mean()
height_median = df["HEIGHT"].median()

print(f"MEAN: {height_mean} MEDIAN: {height_median}")

A propósito, você pode aproximar a distribuição usando uma [estimativa de densidade do kernel (KDE)](https://pandas.pydata.org/docs/reference/api/pandas.Series.plot.density.html). 

In [ ]:
df["HEIGHT"].plot.kde(xlim=(0,50_000))

## Variável de Fase de Voo

Em relação à `HEIGHT`, vamos analisar a `PHASE_OF_FLIGHT`. Para contextualizar, aqui está um ciclo típico que visualiza as fases do voo. Observe que, dependendo da aeronave e da natureza do voo, algumas etapas serão diferentes. Por exemplo, um `EN ROUTE` é típico para um voo que vai do ponto A ao ponto B. Mas se um piloto estiver praticando circuitos em um avião (decolando e pousando repetidamente), isso é chamado de `LOCAL`, pois um padrão local está sendo executado.

![](https://github.com/thomasnield/anaconda_python_eda/raw/public/resource/7Od2TS0O.svg)


Devemos esperar que as fases do voo mais próximas do solo tenham mais colisões com pássaros, com base em nossa análise anterior da variável `HEIGHT`. Vamos dar uma olhada e plotar o `value_counts()` como um gráfico de barras.

In [ ]:
df["PHASE_OF_FLIGHT"].value_counts().plot.bar()

Portanto, não há nada de surpreendente aqui. Fases do voo mais próximas do solo apresentam mais colisões com pássaros. Como essa variável é discreta, pode ser útil observar o **modo**, o(s) valor(es) que ocorre(m) com mais frequência. Podemos ver que `Approach` é o modo, o que significa que é a fase do voo mais comum para colisões com pássaros.

In [ ]:
df["PHASE_OF_FLIGHT"].mode()

## Variável de Velocidade

A seguir, vamos dar uma olhada em `SPEED`. Quanto mais rápido um avião estiver voando, maior a probabilidade de ele ser danificado ao colidir com um pássaro, resultando em um relatório de colisão com pássaros. Um pássaro que colide com um avião em movimento lento tem menos probabilidade de ser considerado uma colisão com pássaros se nenhum dano ocorrer, certo? No entanto, um motor girando em uma aeronave parada pode sugar um pássaro e certamente também ser considerado uma colisão com pássaros.

Vamos dar uma olhada.

In [ ]:
df["SPEED"].hist(bins=50)

Parece que temos uma distribuição normal aqui, como indicado pela curva em forma de sino, com alguns valores extremos à direita. Isso vai ser interessante. Vamos calcular a média e a mediana disso.

In [ ]:
speed_mean = df["SPEED"].mean()
speed_median = df["SPEED"].median()

print(f"MEAN: {speed_mean} MEDIAN: {speed_median}")

Com certeza, nossa média não está muito longe da nossa mediana, então obtivemos uma variável com boa aparência e algum valor preditivo. E, novamente, isso pode fazer sentido. Quando uma aeronave está se movendo lentamente, ela não está se movendo rápido o suficiente para que um pássaro a atinja de forma prejudicial (a menos que seja sugado por um motor). Se estiver se movendo rapidamente, provavelmente está em altitude de cruzeiro alta e longe de onde as aves são encontradas. Pode haver uma correlação até mesmo entre velocidade e altura, que exploraremos na próxima seção.

Para garantir, vamos aproximar a distribuição de probabilidade. Se usarmos velocidade para certas tarefas, podemos considerar cortar os valores atípicos naquela cauda direita.

In [ ]:
df["SPEED"].plot.kde(xlim=(0,2000))

## Valores Atípicos

**Valores atípicos** são valores que estão muito distantes da maioria dos valores em uma distribuição. A maneira como lidamos com valores atípicos depende do que estamos tentando fazer e do contexto do problema. Podemos removê-los, substituí-los ou simplesmente deixá-los como estão, dependendo do que o valor atípico significa para o problema em questão.

Embora existam casos válidos para remover valores atípicos, lembre-se de perguntar o que eles significam em sua aplicação. Seu termostato inteligente pode não precisar aprender com um dia excepcionalmente frio de maio, e esse é um valor atípico que você pode considerar remover com segurança. No entanto, um pedestre fantasiado de galinha atrapalhando a visão computacional de um carro "autônomo" é um problema muito sério, mesmo que seja um valor atípico. Não queremos remover isso, pois indica que temos problemas maiores com nosso domínio.

Valores atípicos são um tópico muito difícil de acertar e exigem não apenas um entendimento de estatística, mas também um entendimento do problema. Lembre-se disso!

### Intervalo Interquartil (IQR) e Percentis

Lembre-se de que a maioria das colisões com pássaros ocorreu bem antes de 10.00 pés, o que distorce bastante os dados para a esquerda.

In [ ]:
df["HEIGHT"].hist(bins=30)

Vamos analisar os registros em que as colisões com pássaros excederam essa altura e hipotetizá-los como outliers.

In [ ]:
with pd.option_context('display.max_columns', None):
    display(df[df["HEIGHT"] > 10_000])

Certo, 325 linhas é uma quantidade relativamente pequena em comparação com todo o conjunto de dados. Enquanto isso entra na análise bivariada, vamos saciar nossa curiosidade e perguntar quais espécies de pássaros são capazes de voar tão alto, de acordo com os dados.

In [ ]:
df[df["HEIGHT"] > 10_000]["SPECIES"].value_counts(dropna=False)

Certo, muitas aves desconhecidas e muita diversidade sem um padrão claro. Há alguma ave voando acima de 25.000 pés?

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df[df["HEIGHT"] > 25_000])

Interessante. Tivemos apenas 3 ocorrências em que colisões com pássaros ocorreram acima de 7.660 metros, incluindo uma "andorinha-do-penhasco" e uma "toutinegra-de-wilson". É possível que pássaros consigam voar tão alto? Se fizermos alguma pesquisa, o maior registro de colisão entre pássaros foi em 1973, quando [um abutre colidiu a 37.000 pés](https://sora.unm.edu/sites/default/files/journals/wilson/v086n04/p0461-p0462.pdf). 

Vamos formalizar um pouco mais nossa análise. Como vimos, `HEIGHT` não é um daqueles casos que seguem a curva em sino da distribuição normal. Outra maneira de abordar outliers nesses casos é usar o método do **Amplitude Interquartil (IQR)**. O **IQR** é a diferença entre o 75º e o 25º percentil. Ao nos referirmos aos percentis trimestrais (0, 25, 50, 75 e 100), nos referimos a eles como quartis. Um quartil de 50% seria o valor mais central (a mediana), ou a média dos dois valores mais centrados.

Um gráfico de caixa (também chamado de "gráfico de caixa de bigodes") visualizará tudo isso rapidamente, como mostrado abaixo.

<img src ="https://github.com/thomasnield/anaconda_python_eda/raw/public/resource/8U7f1C6A.png" width="600"> </img>

O valor `1,5` é conhecido como $ k $, e podemos aumentá-lo para elevar o limite para o que consideramos um *outlier*. O gráfico de caixa não mostrará apenas o intervalo dos dados, mas também para onde a maioria dos dados gravita e sua assimetria. Vamos mostrar um `boxplot()` para `HEIGHT`.

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt

sns.boxplot(x=df['HEIGHT'])

Bem... isso é um pouco confuso. Os 25% superiores dos valores estão acima de 1.000 pés e se espalham por mais de 30.000 pés. Os 25% inferiores dos valores estão extremamente comprimidos a 0 metros, assim como todos os valores discrepantes. Isso é verdade? Vamos obter esses números exatos. Vamos também remover os `NaN`'s aqui, porque elas não fornecerão valor e desviarão a atenção dos valores que temos. Embora valores não relatados possam ser problemáticos, vamos determinar se é aceitável removê-los.

In [ ]:
from numpy import percentile

q25 = percentile(df["HEIGHT"].dropna(), 25)
q75 = percentile(df["HEIGHT"].dropna(), 75)

q25, q75

Portanto, os 25% inferiores dos valores estão de fato ao nível do solo, a 0 pés. De fato, 44% dos valores de `HEIGHT` registrados ocorrem ao nível do solo. Podemos calcular isso assim:

In [ ]:
sum(df["HEIGHT"] == 0) / df["HEIGHT"].dropna().shape[0]

Isso pode fazer sentido, já que as aves tendem a ficar perto do solo, onde estão alimentos, ninhos, água, locais de descanso e outros habitats essenciais.

Vamos fazer a mesma proporção para pelo menos 1.000 pés. Com certeza, 26% dos valores estão acima de 1.000 pés.

In [ ]:
sum(df["HEIGHT"] >= 1000) / df["HEIGHT"].dropna().shape[0]

Então, o que poderíamos considerar como outliers? Vamos tentar quaisquer valores que excedam $ Q1 \pm 1,5 \times \text{IQR} $. Esse `1,5` serviria como o valor `k` inicial, e podemos aumentá-lo para um limite de outlier mais alto, se necessário (por exemplo, estamos obtendo "muitos" outliers).

In [ ]:
iqr = q75 - q25
k = 1.5
cut_off = iqr * k
lower = q25 - cut_off
upper = q75 + cut_off

outliers = df[(df['HEIGHT'] < lower) | (df['HEIGHT'] > upper)]

with pd.option_context('display.max_columns', None):
    display(outliers)

Não podemos dizer que seja útil detectar outliers na direção inferior, visto que os 0s dominam qualquer coisa abaixo do 44º percentil, então eles não são realmente outliers. Mas a direção superior pode ser útil, então vamos nos concentrar apenas nessa direção. Vamos aumentar o valor `k` para `10` porque realmente queremos aumentar o limite para ver valores verdadeiramente marginais.

In [ ]:
iqr = q75 - q25
k = 10
cut_off = iqr * k
upper = q75 + cut_off

outliers = df[(df['HEIGHT'] > upper)]

with pd.option_context('display.max_columns', None):
    display(outliers)

Com um limite tão alto, obtemos 219 outliers. Analisando os dados, parece haver muitos aviões comerciais pesados ​​pilotados pela UNITED AIRLINES e pela SOUTHWEST AIRLINES. Correndo o risco de entrar em análise bivariada, vamos dar uma olhada nas `AIRCRAFT` nesses valores discrepantes para testar essa teoria.

In [ ]:
outliers["AIRCRAFT"].value_counts(dropna=False)

Ok, interessante... ou talvez não! As companhias aéreas voam com aeronaves grandes como o 737-800 em altitudes muito altas e com bastante frequência. E onde há mais frequência, há mais oportunidade de observar valores discrepantes, como aeronaves colidindo com pássaros em altitudes mais elevadas. Talvez a [Lei dos Números Verdadeiramente Grandes](https://en.wikipedia.org/wiki/Law_of_truly_large_numbers) esteja desempenhando um papel aqui `¯\_(ツ)_/¯`.

## Valores atípicos com desvio padrão

Como nossa variável `SPEED` parece seguir uma distribuição normal, podemos detectar valores atípicos usando desvios-padrão.

Vamos criar outro boxplot.

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt

sns.boxplot(x=df['SPEED'])

Ok, isso está bastante equilibrado. Há alguns valores atípicos no lado direito, mas não muitos. Vamos focar nossa atenção nessa direção correta, especificamente valores maiores que 3 desvios-padrão da média.

In [ ]:
speed_mean = df["SPEED"].mean()
speed_std = df["SPEED"].std()
outliers = df[df["SPEED"] > (speed_mean+speed_std*3)]

with pd.option_context('display.max_columns', None):
    display(outliers)

Isso nos deixa com 129 registros. Exploraremos isso em relação a outras variáveis, como o tipo de aeronave e o porta-aviões, na próxima seção. Vamos olhar na direção oposta, mas há um problema: 3 desvios-padrão à esquerda da média são negativos e não temos velocidades negativas registradas.

In [ ]:
print(speed_mean-speed_std*3)

Vamos voltar para 2,5 desvios-padrão.

In [ ]:
print(speed_mean-speed_std*2.5)

In [ ]:
outliers = df[df["SPEED"] < (speed_mean-speed_std*2.9)]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(outliers)

Humm, muitas dessas aeronaves voando tão devagar que foram capturadas como valores atípicos parecem estar no solo. Faz sentido. Deixaremos essa análise bivariada para a próxima seção para um mergulho mais profundo.

## EXERCISE

Explore as variáveis ​​`DISTANCE` (que são milhas náuticas do aeroporto) e `AC_CLASS`. O que você pode observar sobre cada uma delas?

Para contextualizar, `AC_CLASS` é decodificado na tabela a seguir:

| Código da Aeronave | Classificação da Aeronave |
|--------------------|---------------------------|
| A                  | Avião                     |
| B                  | Helicóptero               |
| C                  | Planador                  |
| D                  | Balão                     |
| F                  | Dirigível                 |
| I                  | Giroplano                 |
| J                  | Ultraleve                 |
| Y                  | Outro                     |
| Z                  | Desconhecido              |

In [ ]:
# coloque seu código aqui


### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

Usando um histograma e um gráfico do KDE, podemos ver que as colisões com pássaros têm uma forte concentração perto do aeroporto.

In [ ]:
df["DISTANCE"].hist(bins=30)

In [ ]:
df["DISTANCE"].plot.kde(xlim=(0,150))

Com `AC_CLASS`, as colisões com pássaros ocorrem predominantemente com aviões (classe `A`), seguidos por helicópteros (classe `B`). Isso faz sentido, já que planadores e ultraleves são provavelmente menos comuns, e não porque aeronaves e helicópteros sejam mais vulneráveis ​​a colisões com pássaros.

In [ ]:
df["AC_CLASS"].value_counts().plot.bar()